In [110]:
#%pip install -q tensorflow-recommenders
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Embedding, Dense
from tensorflow.keras import Sequential
import tensorflow_recommenders as tfrs
from tensorflow.keras.metrics import AUC
import math
import numpy as np



In [111]:
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [112]:
# Load data
basepath = '../../datasets/preprocessed_datasets/gabor/'
pandas_interaction_data = pd.read_pickle(basepath + 'user_item_interactions_only_main_products.pkl')
pandas_interaction_data = pandas_interaction_data.sample(1000)
pandas_interaction_data.amount = pandas_interaction_data.amount.apply(lambda x: 1 if x > 0 else 0) 
product_data = pd.read_pickle(basepath + 'no_product_variants_with_duplicate_ids.pkl')
pandas_interaction_data.main_product_id = pandas_interaction_data.main_product_id.astype(int)
product_data.main_product_id = product_data.main_product_id.astype(int)

display(pandas_interaction_data, product_data)

,customer_id,main_product_id,amount
102125,3861599,464697,1
101525,3816902,360839,1
416434,8907217,4775539,1
5597,1957380,3866469,1
442671,9280479,5337911,1
...,...,...,...
562285,10821792,4064261,1
115656,4142856,3476448,1
244708,6084583,5336256,1
539849,10505679,3696391,1


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes
0,557559,4058394021466,sportliche Ballerinas Glattleder schwarz,schwarz,02.643.57,6590678,315571,w,PT,F-S|H-W,...,bis 3 cm,40.0,6.5,11.0,0.0,Gummi,0.0,0.0,"[557559, 549733, 549734, 557553, 589488, 58948...","[35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40...."
1,551622,4054452768212,elegante Pumps Glattleder schwarz,schwarz,05.160.37,6590677,315573,w,PT,F-S|H-W,...,3 cm - 5 cm,38.0,5.0,8.0,0.0,EVA,0.0,0.0,"[551622, 551505, 551510, 551509, 552973, 55162...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
2,547193,4054452768427,Slipper Glattleder schwarz,schwarz,04.443.27,6590677,315576,w,SK,F-S|H-W,...,3 cm - 5 cm,42.0,8.0,14.0,0.0,PU-TPU,0.0,0.0,"[547193, 547191, 547196, 547187, 547189, 54719...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
3,549776,4059701687894,Sneaker low Rauleder blau,blau,06.968.46,4997827,315567,w,VN,F-S|H-W,...,3 cm - 5 cm,39.0,6.0,10.0,0.0,Gummi-EVA,0.0,0.0,"[549776, 550599, 550596, 550593, 550601, 54978...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
4,550726,4054452851594,elegante Ballerinas Materialmix Lederimitat sc...,schwarz,06.102.67,6590678,315574,w,SK,F-S|H-W,...,bis 3 cm,37.5,4.5,7.0,0.0,EVA,0.0,0.0,"[550726, 550736, 550735, 550732, 550728, 55072...","[35.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40.0, 40...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3292,10603553,4066558951733,Sandale met plateauzool Suède blauw,blauw,24.764.36,6590677,315568,w,SK,F-S,...,5 cm - 8 cm,35.0,2.5,3.0,0.0,PU-TPU,0.0,0.0,"[10603553, 10544662]","[35.0, 37.0]"
3293,10442948,4065171827272,Mokassin Materialmix Leder pink,pink,26.090.21,6590678,315590,w,PT,F-S,...,bis 3 cm,40.5,7.0,12.0,0.0,Gummi,0.0,0.0,"[10442948, 10442949]","[40.5, 41.0]"
3294,10782704,4251234499207,Shopper ANDIE blau,blau,921453,363013,363017,w,DE,F-S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[10782704],[]
3295,10679703,4066558303617,Elegante pumps Glad leer wit,wit,21.450.60,6590677,315573,w,PT,F-S,...,5 cm - 8 cm,35.0,2.5,3.0,0.0,TPU,0.0,0.0,[10679703],[35.0]


In [113]:
# Create a tf.data.Dataset from the interaction data
interaction_dataset = tf.data.Dataset.from_tensor_slices({
    'customer_id': pandas_interaction_data['customer_id'].values,
    'main_product_id': pandas_interaction_data['main_product_id'].values,
    'amount': pandas_interaction_data['amount'].values
})
#variables 
dataset_len = pandas_interaction_data.shape[0]
test_len = math.ceil(dataset_len * 0.2)
train_len = dataset_len - test_len

metrics_batchsize = 16
train_batch_size = 128
test_batch_size = 64
random_seed = 27

In [114]:
#train test split
tf.random.set_seed(random_seed)
# shuffled = interaction_dataset.shuffle(len, seed=random_seed, reshuffle_each_iteration=False)
# train = shuffled.take(train_len)
# test = shuffled.skip(train_len).take(test_len)
shuffled = interaction_dataset.shuffle(dataset_len, seed=random_seed, reshuffle_each_iteration=False)

train = shuffled.take(train_len)
test = shuffled.skip(train_len).take(test_len)
display(train, test)

<_TakeDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'main_product_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'amount': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

<_TakeDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'main_product_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'amount': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

In [115]:
# Unique customer and product identifiers
unique_customer_ids = np.array(pandas_interaction_data["customer_id"].unique())
unique_product_ids = np.array(pandas_interaction_data["main_product_id"].unique())
user_ids_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
user_ids_vocabulary.adapt(unique_customer_ids)

product_ids_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
product_ids_vocabulary.adapt(unique_product_ids)
# print(unique_product_ids.shape[0], unique_customer_ids.shape[0], unique_product_ids, unique_product_ids)

In [116]:
# Define a model using TensorFlow Recommenders
product_ids_dataset = tf.data.Dataset.from_tensor_slices(unique_product_ids)
class RetailModel(tfrs.Model):

    def __init__(self, unique_product_ids_vocab, unique_customer_ids_vocab, embedding_dimension=32):
        super().__init__()
        # Set up user and product representations
        self.user_embedding = tf.keras.Sequential([
            unique_customer_ids_vocab,
            Embedding(unique_customer_ids_vocab.vocabulary_size(), embedding_dimension)
        ])
        self.product_embedding = tf.keras.Sequential([
            unique_product_ids_vocab,
            Embedding(unique_product_ids_vocab.vocabulary_size(), embedding_dimension)
        ])
        # Set up a dense layer for the task.
        self.dense_layer = Dense(128, activation="relu")
        


        # Set up retrieval task and metrics
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(candidates=product_ids_dataset.batch(metrics_batchsize).map(self.product_model))
        )
        self.auc_metric = AUC(name='auc')
        self.rmse_metric = tf.keras.metrics.RootMeanSquaredError(name='rmse')
        self.precision = tf.keras.metrics.Precision(name='precision')
        self.recall = tf.keras.metrics.Recall(name='recall')

    def product_model(self, product_ids):
        return self.product_embedding(product_ids)

    def dot_product_score(self, user, product):
        """
        Computes the dot product between user and product embeddings to get the interaction score.
        """
        return tf.reduce_sum(user * product, axis=1)

    def compute_loss(self, features, training=False):
        # print(features)
        user_embeddings = self.user_embedding(features["customer_id"])
        product_embeddings = self.product_embedding(features["main_product_id"])
        
        user_output = self.dense_layer(user_embeddings)
        product_output = self.dense_layer(product_embeddings)
        # This is a hypothetical function that returns the logits or scores
        # This needs to be adapted based on your actual model
        positive_logits = self.dot_product_score(user_output, product_output)

        # Update RMSE
        self.rmse_metric.update_state(y_true=features["amount"], y_pred=positive_logits)


        return self.task(user_output, product_output)

    def evaluate(self, validation_dataset, *args, **kwargs):
        # Call the base class's evaluate method
        
    
        # Compute additional metrics
        for features in validation_dataset:
            user_embeddings = self.user_embedding(features["customer_id"])
            product_embeddings = self.product_embedding(features["main_product_id"])
            
            user_output = self.dense_layer(user_embeddings)
            product_output = self.dense_layer(product_embeddings)
            
            positive_logits = self.dot_product_score(user_output, product_output)
            # Assuming you have a binary "label" in your dataset indicating 1 for positive interaction and 0 for negative
            self.precision.update_state(y_true=features["amount"], y_pred=positive_logits)
            self.recall.update_state(y_true=features["amount"], y_pred=positive_logits)
    
        precision_result = self.precision.result().numpy()
        recall_result = self.recall.result().numpy()
    
        # Calculate F1 score
        if (precision_result + recall_result) != 0:
            f1_score = 2 * (precision_result * recall_result) / (precision_result + recall_result)
        else:
            f1_score = 0.0
    
        # Reset the metrics for the next evaluation
        # self.precision.reset_states()
        # self.recall.reset_states()
        base_results = super(RetailModel, self).evaluate(validation_dataset, *args, **kwargs)
    
        return base_results, [precision_result, recall_result, f1_score]  # or append additional results as needed



In [117]:
model = RetailModel(product_ids_vocabulary, user_ids_vocabulary, embedding_dimension=128)
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001)) # using legacy instead of tf.keras.optimizers.Adagrad, because newer version is slow on m1/m2 macs

In [118]:
# Shuffle, batch, and cache the data.
cached_train = train.shuffle(dataset_len).batch(train_batch_size).cache()
cached_test = test.batch(test_batch_size).cache()
# Train the model
# model.fit(cached_train, validation_data=cached_test, validation_freq=5, epochs=3)
model.fit(cached_train, validation_freq=5, epochs=6)

Epoch 1/6
7/7 [==============================] - 4s 492ms/step - auc: 0.0000e+00 - rmse: 1.0003 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.0225 - factorized_top_k/top_5_categorical_accuracy: 0.0538 - factorized_top_k/top_10_categorical_accuracy: 0.0800 - factorized_top_k/top_50_categorical_accuracy: 0.1937 - factorized_top_k/top_100_categorical_accuracy: 0.2988 - loss: 493.5778 - regularization_loss: 0.0000e+00 - total_loss: 493.5778
Epoch 2/6
7/7 [==============================] - 3s 439ms/step - auc: 0.0000e+00 - rmse: 0.9793 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.2925 - factorized_top_k/top_5_categorical_accuracy: 0.4475 - factorized_top_k/top_10_categorical_accuracy: 0.5075 - factorized_top_k/top_50_categorical_accuracy: 0.7063 - factorized_top_k/top_100_categorical_accuracy: 0.8225 - loss: 491.5560 - regularization_loss: 0.0000e+00 - total_loss: 491.5560
Epoch 3/6
7/7 [=======

In [119]:
# Evaluate the model
k = 5
result_evaluate_train = model.evaluate(cached_train)
result_evaluate_train

7/7 [==============================] - 3s 439ms/step - auc: 0.0000e+00 - rmse: 0.6308 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 1.0000 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 471.5708 - regularization_loss: 0.0000e+00 - total_loss: 471.5708


([0.0,
  0.6307846307754517,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  104.50016784667969,
  0,
  104.50016784667969],
 [1.0, 0.05625, 0.1065088753222601])

In [120]:
result_evaluat_test = model.evaluate(cached_test)
result_evaluat_test

4/4 [==============================] - 2s 455ms/step - auc: 0.0000e+00 - rmse: 0.8556 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.9850 - factorized_top_k/top_5_categorical_accuracy: 0.9900 - factorized_top_k/top_10_categorical_accuracy: 0.9900 - factorized_top_k/top_50_categorical_accuracy: 0.9950 - factorized_top_k/top_100_categorical_accuracy: 0.9950 - loss: 166.5064 - regularization_loss: 0.0000e+00 - total_loss: 166.5064


([0.0,
  0.8555887937545776,
  0.0,
  0.0,
  0.9850000143051147,
  0.9900000095367432,
  0.9900000095367432,
  0.9950000047683716,
  0.9950000047683716,
  16.606243133544922,
  0,
  16.606243133544922],
 [0.0, 0.0, 0.0])

In [121]:
print("Name\t\tValue")
print("-" * 30)
for metric in model.metrics:
    print(f"{metric.name}\t\t{metric.result().numpy()}")
print(f"AUC: {model.auc_metric.result().numpy()}")

Name		Value
------------------------------
auc		0.0
rmse		0.8555887937545776
precision		0.0
recall		0.0
factorized_top_k/top_1_categorical_accuracy		0.9850000143051147
factorized_top_k/top_5_categorical_accuracy		0.9900000095367432
factorized_top_k/top_10_categorical_accuracy		0.9900000095367432
factorized_top_k/top_50_categorical_accuracy		0.9950000047683716
factorized_top_k/top_100_categorical_accuracy		0.9950000047683716
AUC: 0.0


In [ ]:
sample = next(iter(cached_test.take(1)))

# Extract customer_id from the sample
customer_id = sample['customer_id'].numpy()
customer_id

In [144]:
def get_random_customer_from_test_data():
    return next(iter(test.take(1)))['customer_id'].numpy()

def display_item_ids(item_ids):
    display(product_data[product_data.main_product_id.isin(item_ids) ])

def display_products_by_customer_id(customer_id):
    display_item_ids(pandas_interaction_data[pandas_interaction_data.customer_id ==  customer_id].main_product_id.tolist())
    

index = tfrs.layers.factorized_top_k.BruteForce(model.user_embedding)
index.index_from_dataset(
    product_ids_dataset.batch(100).map(lambda id: (id, model.product_model(id))))

def predict_user(user_id):
    print('predicting user: ', user_id)
    print('user already bought following products: ')
    display_products_by_customer_id(user_id)
    
    score, predicted_product_ids = index(np.array([user_id]))
    
    print('predicted products: ')
    display_item_ids(predicted_product_ids[0].numpy())
    print('scores: ', score[0].numpy())


user_id = get_random_customer_from_test_data()

predict_user(user_id)

predicting user:  7371834
user already bought following products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes
1398,6570268,4064032835586,Biker- / Combat Boot Effektleder schwarz,schwarz,71.730.37,6590677,5014852,w,PT,H-W,...,5 cm - 8 cm,43.0,9.0,16.0,17.0,PU-TPU,0.0,0.0,"[6570268, 6570269, 6685292, 6685295, 6685290, ...","[35.0, 35.5, 36.0, 37.5, 38.0, 38.5, 39.0, 40...."


predicted products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes
57,3504613,4062862503521,Sneaker low Rauleder rot,rot,56.498.38,6590678,315567,w,PT,H-W,...,3 cm - 5 cm,42.5,8.5,15.0,0.0,EVA,0.0,0.0,"[3504613, 3473297, 3504603, 3473298, 3473464, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
253,3788595,4062862982111,Sneaker high Rauleder rosa,rosa,53.736.14,6590677,315586,w,SK,H-W,...,3 cm - 5 cm,40.0,6.5,11.0,9.0,TR,0.0,0.0,"[3788595, 3788599, 3788594, 3788597, 3788596, ...","[35.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40.0, 40...."
310,3867012,4062862326571,Sneaker low Materialmix Leder grün,grün,56.358.83,6590678,315567,w,SK,H-W,...,3 cm - 5 cm,42.0,8.0,14.0,0.0,TR,0.0,0.0,"[3867012, 3867008, 3867006, 3867015, 3867011, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
598,4774679,4062862829751,Keilsandalette Rauleder gelb,gelb,62.750.22,6590678,315564,w,PT,F-S,...,3 cm - 5 cm,35.0,2.5,3.0,0.0,TR,0.0,1.0,"[4774679, 4773177, 4774684, 4773750, 4773755, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
714,4979174,4062862767572,Sneaker low Materialmix Leder/Lederimitat mult...,multicolour,66.365.31,6590678,315567,w,VN,F-S,...,3 cm - 5 cm,41.0,7.5,13.0,0.0,Gummi-EVA,0.0,0.0,"[4979174, 4979177, 4979169, 4979170, 4979171, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
865,5135470,4064032008614,Slipper Rauleder beige,beige,62.433.30,6590678,315576,w,PT,F-S,...,bis 3 cm,44.0,9.5,17.0,0.0,TR,0.0,0.0,"[5135470, 5135456, 5135462, 5149514, 5149515, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
982,5015102,4062862740254,PG1018 Sneaker low Materialmix Leder/Textil grün,grün,1018.11.04,3692257,3692261,m,VN,F-S,...,3 cm - 5 cm,43.0,9.0,9.0,0.0,Gummi-EVA,0.0,0.0,"[5015102, 5015308, 5015303, 5015304, 5015306, ...","[39.0, 40.0, 40.5, 41.0, 42.0, 42.5, 43.0, 44...."
1278,6115448,4064032454466,Biker- / Combat Boot Rauleder schwarz,schwarz,72.704.47,6590678,5014852,w,PT,H-W,...,3 cm - 5 cm,44.0,9.5,17.0,18.0,TR,0.0,0.0,"[6115448, 6115441, 6115442, 6467634, 6115443, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
1781,7594101,4064032162385,Eleganter Ballerina Rauleder rosa,rosa,81.302.10,6590677,315576,w,PT,F-S,...,bis 3 cm,37.5,4.5,7.0,0.0,TPU,0.0,0.0,"[7594101, 7594104, 7594103, 7594106, 7594100, ...","[35.0, 35.5, 37.0, 37.5, 38.0, 38.5, 39.0, 40...."
2513,8502314,4065171188113,Sportlicher Stiefel Glattleder beige,beige,91.739.21,6590677,5014850,w,PT,H-W,...,5 cm - 8 cm,38.5,5.5,9.0,42.0,PU-TPU,0.0,0.0,"[8502314, 8502322, 8502321, 8502317, 8502310, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."


scores:  [0.02840061 0.02751753 0.02736414 0.0265623  0.02621533 0.02588076
 0.02574887 0.02568691 0.02432163 0.02376205]
